In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
from matplotlib.ticker import NullFormatter
import seaborn as sns
import math
import os
from matplotlib import rcParams
from datetime import datetime
from sklearn.utils import shuffle
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.preprocessing import LabelEncoder 
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import joblib

In [3]:
content1_8_final= pd.read_csv("/content/drive/My Drive/Colab Notebooks/courseraforums-master/data/content1_8_final.csv",encoding = 'unicode_escape')

In [4]:
#Frequency distribution of classes"
content1_8_final_outcome = pd.crosstab(index=content1_8_final["forum_id"],  # Make a crosstab
                              columns="count")      # Name the count column

content1_8_final_outcome

col_0,count
forum_id,
2,173438
3,193276
4,62781
7,18273
8,111039
9,18220
10,162047


In [5]:
content1_8_final = content1_8_final.replace({'forum_id': {2: 'General Discussion', 3: 'Assignments', 4:'Study Group', 7:'Feedback', 8:'Lectures', 9:'Technical Issues', 10:'Other'}})
#Frequency distribution of classes"
content1_8_final_outcome = pd.crosstab(index=content1_8_final["forum_id"],  # Make a crosstab
                              columns="count")      # Name the count column

content1_8_final_outcome

col_0,count
forum_id,
Assignments,193276
Feedback,18273
General Discussion,173438
Lectures,111039
Other,162047
Study Group,62781
Technical Issues,18220


In [6]:
content1_8_final = content1_8_final.drop(['course_id'], axis=1)

In [7]:
content1_8_final = shuffle(content1_8_final)

In [8]:
train = content1_8_final.iloc[20000:25000,:] # train data
test = content1_8_final.iloc[35000:40000,:] # test data

In [9]:
print("Any missing sample in training set:",train.isnull().values.any())
print("Any missing sample in test set:",test.isnull().values.any(), "\n")

Any missing sample in training set: False
Any missing sample in test set: False 



In [10]:
#Seperating Predictors and Outcome values from train and test sets
X_train = pd.DataFrame(train.drop(['forum_id'],axis=1))
print(X_train)
Y_train_label = train.forum_id.values.astype(object)
print(Y_train_label)
X_test = pd.DataFrame(test.drop(['forum_id'],axis=1))
Y_test_label = test.forum_id.values.astype(object)

        thread_id  no_of_posts  ...  votes_Square  index_max_votes
434885        191         19.0  ...             9           434864
676951       6135          1.0  ...             1           676951
512841        215         26.0  ...             0           512822
469618         45          8.0  ...             4           469618
330893        110        343.0  ...             0           330489
...           ...          ...  ...           ...              ...
283979       2084          2.0  ...             0           283979
169450        211         10.0  ...             0           169438
62866         137          3.0  ...             1            62863
369234       3700          6.0  ...             0           369236
9014          307          3.0  ...             4             9014

[5000 rows x 18 columns]
['Lectures' 'General Discussion' 'Lectures' ... 'Assignments' 'Other'
 'Other']


In [11]:
# Dimension of Train and Test set 
print("Dimension of Train set",X_train.shape)
print("Dimension of Test set",X_test.shape,"\n")

Dimension of Train set (5000, 18)
Dimension of Test set (5000, 18) 



In [12]:
# Transforming non numerical labels into numerical labels
from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()

# encoding train labels 
# encoder.fit(Y_train_label)
# Y_train = encoder.transform(Y_train_label)
# # encoding test labels 
# encoder.fit(Y_test_label)
# Y_test = encoder.transform(Y_test_label)

#Total Number of Continous and Categorical features in the training set
num_cols = X_train._get_numeric_data().columns
print("Number of numeric features:",num_cols.size)
#list(set(X_train.columns) - set(num_cols))
names_of_predictors = list(X_train.columns.values)

# Scaling the Train and Test feature set 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Number of numeric features: 18


In [13]:
#Libraries to Build Ensemble Model : Random Forest Classifier 
# Create the parameter grid based on the results of random search 
params_grid = [{'kernel': ['rbf'], 'gamma': [1e-3],
                     'C': [10]},
                    {'kernel': ['linear'], 'C': [10]}]

In [15]:
#Performing CV to tune parameters for best SVM fit 
svm_model = GridSearchCV(SVC(), params_grid, cv=5)
svm_model.fit(X_train_scaled, Y_train_label)

GridSearchCV(cv=5, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [10], 'gamma': [0.001], 'kernel': ['rbf']},
                         {'C': [10], 'kernel': ['linear']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [17]:
# save the model to disk
filename = '/content/drive/My Drive/Colab Notebooks/courseraforums-master/data/finalized_model.sav'
joblib.dump(svm_model, filename)

['/content/drive/My Drive/Colab Notebooks/courseraforums-master/data/finalized_model.sav']

In [20]:
# View the accuracy score
print('Best score for training data:', svm_model.best_score_,"\n") 

# View the best parameters for the model found using grid search
print('Best C:',svm_model.best_estimator_.C,"\n") 
print('Best Kernel:',svm_model.best_estimator_.kernel,"\n")
print('Best Gamma:',svm_model.best_estimator_.gamma,"\n")

final_model = svm_model.best_estimator_
Y_pred_label = final_model.predict(X_test_scaled)
#Y_pred_label = list(encoder.inverse_transform(Y_pred))

Best score for training data: 0.4436 

Best C: 10 

Best Kernel: linear 

Best Gamma: scale 



In [21]:
print(len(Y_test_label))
print(len(Y_pred_label))

5000
5000


In [23]:
#Making the Confusion Matrix
print(confusion_matrix(Y_test_label,Y_pred_label))
print("\n")
print(classification_report(Y_test_label,Y_pred_label))

print("Training set score for SVM: %f" % final_model.score(X_train_scaled , Y_train_label))
print("Testing  set score for SVM: %f" % final_model.score(X_test_scaled  , Y_test_label ))

svm_model.score

[[1164    0   60   31   49    6    0]
 [  96    0    4    1   13    3    0]
 [ 691    0  314   25  123   49    0]
 [ 449    0   94   88   84    8    0]
 [ 433    0  112   33  469   34    0]
 [ 131    0  130   21   15  141    0]
 [ 100    0    4    3   18    4    0]]


                    precision    recall  f1-score   support

       Assignments       0.38      0.89      0.53      1310
          Feedback       0.00      0.00      0.00       117
General Discussion       0.44      0.26      0.33      1202
          Lectures       0.44      0.12      0.19       723
             Other       0.61      0.43      0.51      1081
       Study Group       0.58      0.32      0.41       438
  Technical Issues       0.00      0.00      0.00       129

          accuracy                           0.44      5000
         macro avg       0.35      0.29      0.28      5000
      weighted avg       0.45      0.44      0.39      5000



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training set score for SVM: 0.455200
Testing  set score for SVM: 0.435200


<bound method BaseSearchCV.score of GridSearchCV(cv=5, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [10], 'gamma': [0.001], 'kernel': ['rbf']},
                         {'C': [10], 'kernel': ['linear']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)>

In [69]:
# print("Training set score for SVM: %f" % final_model.score(X_train_scaled , Y_train))
# print("Testing  set score for SVM: %f" % final_model.score(X_test_scaled  , Y_test ))

Training set score for SVM: 0.443633
Testing  set score for SVM: 0.442533


In [24]:
#Fitting the XGBOOST model
xg_class = xgb.XGBClassifier(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [25]:
xg_class.fit(X_train_scaled, Y_train_label)

XGBClassifier(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.3, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=10, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [26]:
Y_XG_pred_label = xg_class.predict(X_test_scaled)

In [27]:
Y_XG_pred_label

array(['Assignments', 'General Discussion', 'Other', ..., 'Assignments',
       'Study Group', 'Assignments'], dtype=object)

In [28]:
print(confusion_matrix(Y_test_label,Y_XG_pred_label))
print("\n")
print(classification_report(Y_test_label,Y_XG_pred_label))



[[1147    0  104   23   34    2    0]
 [  72    0   26    8   11    0    0]
 [ 453    0  621   41   45   42    0]
 [ 299    0  125  245   50    4    0]
 [ 286    0   90   24  656   25    0]
 [  61    0   74   26   21  256    0]
 [  76    0   26    7   16    3    1]]


                    precision    recall  f1-score   support

       Assignments       0.48      0.88      0.62      1310
          Feedback       0.00      0.00      0.00       117
General Discussion       0.58      0.52      0.55      1202
          Lectures       0.66      0.34      0.45       723
             Other       0.79      0.61      0.69      1081
       Study Group       0.77      0.58      0.66       438
  Technical Issues       1.00      0.01      0.02       129

          accuracy                           0.59      5000
         macro avg       0.61      0.42      0.43      5000
      weighted avg       0.62      0.59      0.57      5000



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [29]:
print("Training set score for XGBOOST: %f" % xg_class.score(X_train_scaled , Y_train_label))
print("Testing  set score for XGBOOST: %f" % xg_class.score(X_test_scaled  , Y_test_label))

Training set score for XGBOOST: 0.648400
Testing  set score for XGBOOST: 0.585200


In [30]:
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn import metrics

ModuleNotFoundError: ignored